In [65]:
!pip install pandas numpy networkx numpy scikit-learn

In [66]:
import os
import pandas as pd

excel_path = os.path.join('..', 'data', 'Reworkan data 4.xlsx')
data = pd.read_excel(excel_path)

print(type(data))
# Print each column name to check for extra spaces or non-visible characters
for col in data.columns:
    print(f"'{col}'")
    
data.head()

<class 'pandas.core.frame.DataFrame'>
'Article ID'
'Datum'
'Dan'
'Teden'
'Article created'
'Tip medija'
'Podtip medija'
'Media uuid'
'Medij'
'Rubric uuid'
'Rubrika'
'Publisher uuid'
'Medijski založnik'
'Doseg medijskega založnika'
'Media circulation'
'Lokalnost'
'Periodika'
'Medijske teme'
'Naslov objave'
'Država medija'
'Mesto'
'Država'
'Opis'
'Page/dur'
'Število strani'
'Avtor'
'Entity uuid'
'Analizirana entiteta'
'Class uuid'
'Krovne teme'
'Object uuid'
'Teme'
'Obj rate'
'Obj size'
'Obj cash'
'Obj description'
'Quoted source'
'Quoted src rate'
'Ouoted src description'
'Reporting tone rate'
'Picture rate'
'Company mention title rate'
'Company mention subtitle rate'
'Representatives quotes rate'
'Quoted sources - third party'
'Planned publicity rate'
'Primary publicity rate'
'Size'
'Size rate'
'Key messages rate'
'Article reach value'
'Article position value'
'Final rate'
'Cash'
'Whole article value'
'Reach'
'Important'
'Načrtovanost'
'Primarnost objave'
'Naklonjenost tematike'
'Priso

,Article ID,Datum,Dan,Teden,Article created,Tip medija,Podtip medija,Media uuid,Medij,Rubric uuid,...,Pregled objav - citiranost virov,Razredi Indeksa medijske podobe,Doseg,Naklonjenost teme,Osrednja tema,Ključna sporočila,Fotografija,Omemba v naslovu,Omemba v podnaslovu,Tekst
0,001c7622-d40a-11ed-9d48-dfb487f9037c,2023-04-06 00:00:00,Thursday,14,2023-04-06 01:31:33.498,tisk,tisk,2fd717ed-78ba-4f63-b257-cd096acb6bda,Delo,bc228842-98cc-48ea-8be9-0107becb707a,...,Objava z obema tipoma citatov,3. razred (0 - 34 točk),odličen,Nevtralna,NaN,NaN,Vsebuje fotografijo,Omenjeno podjetje,Omenjeno podjetje,"V spomin Prof. dr. Marija Pečan, dr. med. (193..."
1,00732997-d53a-11ed-9d48-dfb487f9037c,2023-04-07 13:45:00,Friday,14,2023-04-07 13:47:41.072,internet,splet,754da261-9aee-4a1a-b9d8-734cd409fabf,Zurnal24.si,e059ba7d-1a91-48f6-ac1c-d7ed703375ba,...,Objava z obema tipoma citatov,"2. razred (34,5 - 59 točk)",odličen,Nevtralna,NaN,NaN,Vsebuje fotografijo,Omenjeno podjetje,Omenjeno podjetje,"Člani Strateškega sveta za prehrano, ki je med..."
2,01443cab-dbc6-11ed-9246-2b5ebef623ad,2023-04-15 21:42:00,Saturday,15,2023-04-15 21:44:58.961,internet,splet,bc20546f-3a11-4061-90c2-2769468cd542,Delo.si,f57a09ac-450d-4df4-bc2b-a2c7f96a4322,...,Objava z obema tipoma citatov,3. razred (0 - 34 točk),nizek,Nevtralna,NaN,NaN,Vsebuje fotografijo,Omenjeno podjetje,Omenjeno podjetje,Člani komisije se niso opredeljevali do znakov...
3,01e591ff-d2b1-11ed-83eb-0bd28cf4c8f0,2023-04-04 00:00:00,Tuesday,14,2023-04-04 08:22:00.122,tisk,tisk,a67b08ee-a757-40ff-8d7e-ab8fc8246eec,Jana,4dd54f38-07e1-448f-b96d-dad85e669834,...,Objava z obema tipoma citatov,3. razred (0 - 34 točk),zelo dober,Nevtralna,NaN,NaN,Vsebuje fotografijo,Omenjeno podjetje,Omenjeno podjetje,"žJanimn jdrawiljfta Unija f 0 , Težave s ščit..."
4,03e9491b-dd47-11ed-9246-2b5ebef623ad,2023-04-17 19:37:00,Monday,16,2023-04-17 19:40:59.639,internet,splet,754da261-9aee-4a1a-b9d8-734cd409fabf,Zurnal24.si,e059ba7d-1a91-48f6-ac1c-d7ed703375ba,...,Objava z obema tipoma citatov,3. razred (0 - 34 točk),nizek,Nevtralna,NaN,NaN,Vsebuje fotografijo,Omenjeno podjetje,Omenjeno podjetje,"""Tudi vsi v zdravstvenem sistemu bodo dobili s..."


Funkcije za pridobivanje že obstoječih vektorjev iz elastike.

In [67]:
import pandas as pd
import numpy

from esdl import Elastika

def embed_elastic(df: pd.DataFrame, col_name: str = 'Tekst', target_col_name: str = 'Embedding') -> pd.DataFrame:
    # Split the DataFrame into batches of 500 rows
    batch_size = 1000
    num_batches = (len(df) + batch_size - 1) // batch_size
    
    responses = pd.DataFrame()
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = (i + 1) * batch_size
        
        batch = df['Article ID'].iloc[start_idx:end_idx]
        
        # Call the remote service for the current batch
        requests = Elastika()
        requests.limit(1000)
        requests.field(['uuid', 'vector_768___textonic_v1'])
        requests.filter_uuid(batch.tolist())
        
        # Collect the responses
        articles = requests.gets('1996-01-01', '2030-01-01')
        result = {
            'Article ID': [],
            'Embedding': []
        }
        for a in articles:
            if 'vector_768___textonic_v1' not in a.data:
                continue
            vect = numpy.array(a.data['vector_768___textonic_v1'])
            result['Article ID'].append(a.uuid)
            result['Embedding'].append(vect)
        print(f'Requested/received/valid vector : {len(batch)}/{len(articles)}/{len(result["Embedding"])}')
        # Convert the collected responses to a DataFrame
        response_df = pd.DataFrame(result)
        responses = pd.concat([responses, response_df])
        
    # Merge the responses back into the original DataFrame
    df = pd.merge(df, responses, on='Article ID', how='inner')
    return df

Kličem vektorizacijo na stolpcu 'Text' in dobim stolpec z vektorjem 'Embedding'

In [68]:
data = embed_elastic(data, 'Tekst', 'Embedding')
data[['Article ID', 'Datum', 'Medij', 'Tekst', 'Embedding']].head(100)

Requested/received/valid vector : 1000/912/911
Requested/received/valid vector : 1000/924/622
Requested/received/valid vector : 1000/912/808
Requested/received/valid vector : 1000/888/74
Requested/received/valid vector : 991/920/319


,Article ID,Datum,Medij,Tekst,Embedding
0,001c7622-d40a-11ed-9d48-dfb487f9037c,2023-04-06 00:00:00,Delo,"V spomin Prof. dr. Marija Pečan, dr. med. (193...","[0.010961635038256645, 0.022462036460638046, -..."
1,00732997-d53a-11ed-9d48-dfb487f9037c,2023-04-07 13:45:00,Zurnal24.si,"Člani Strateškega sveta za prehrano, ki je med...","[0.02656681276857853, 0.061292026191949844, -0..."
2,01443cab-dbc6-11ed-9246-2b5ebef623ad,2023-04-15 21:42:00,Delo.si,Člani komisije se niso opredeljevali do znakov...,"[0.021359758451581, 0.06009818613529205, -0.00..."
3,01e591ff-d2b1-11ed-83eb-0bd28cf4c8f0,2023-04-04 00:00:00,Jana,"žJanimn jdrawiljfta Unija f 0 , Težave s ščit...","[-0.007801717147231102, 0.004865827038884163, ..."
4,03e9491b-dd47-11ed-9246-2b5ebef623ad,2023-04-17 19:37:00,Zurnal24.si,"""Tudi vsi v zdravstvenem sistemu bodo dobili s...","[0.019265947863459587, 0.036640871316194534, -..."
...,...,...,...,...,...
95,34ce443c-d2da-11ed-9f09-af460eec62b1,2023-04-04 13:16:00,Sta.si,Prodaja prehranskih dopolnil je vse bolj razši...,"[0.01800856925547123, 0.02196991629898548, -0...."
96,35d75476-e0da-11ed-9f09-af460eec62b1,2023-04-22 08:50:00,Rtvslo.si,Raziskujemo spanje. Vsi že poznamo spalne cikl...,"[0.006205578334629536, -0.007030373904854059, ..."
97,35e5b4e9-d511-11ed-9f09-af460eec62b1,2023-04-07 08:53:00,Siol.net,"Termometer prikaže, kako vroč je članek. Skupn...","[-0.01259427797049284, -0.04666970670223236, -..."
98,3632728f-d881-11ed-bfff-ed625638e8e4,2023-04-11 15:31:00,24ur.com,SLOVENIJA\n'Parkinsonova bolezen v našem siste...,"[0.02734069526195526, 0.08372970670461655, -0...."


In [69]:
import pandas as pd
import networkx as nx
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
# Assuming your data variable is a DataFrame that you have already prepared
# Continuing from the last step of the previous script

def cluster_louvain(df: pd.DataFrame, col_name: str = 'Embedding', similarity_threshold: float = 0.96):
    print("Clustering ...")
    embeddings = np.array(df[col_name].to_list())
    labels = [0] * len(embeddings)
    x = cosine_similarity(embeddings, embeddings)
    similarity_matrix = x > similarity_threshold
    graph = nx.from_numpy_array(similarity_matrix)
    communities = nx.algorithms.community.louvain_communities(graph, resolution=0.1)
    for community in communities:
        initial_member = min(community)
        for member in community:
            labels[member] = initial_member

    df['Cluster'] = pd.Series(labels)
    print("Clustered")
    return df

# Compute clustering
data = cluster_louvain(data, 'Embedding', 0.92)  # <- similarity_threshold
data = data.drop(columns=['Embedding'])  # remove the vectors from data we don't need them anymore
data[['Article ID', 'Datum', 'Medij', 'Cluster', 'Tekst']].head(100)


Clustering ...
Clustered


,Article ID,Datum,Medij,Cluster,Tekst
0,001c7622-d40a-11ed-9d48-dfb487f9037c,2023-04-06 00:00:00,Delo,0,"V spomin Prof. dr. Marija Pečan, dr. med. (193..."
1,00732997-d53a-11ed-9d48-dfb487f9037c,2023-04-07 13:45:00,Zurnal24.si,1,"Člani Strateškega sveta za prehrano, ki je med..."
2,01443cab-dbc6-11ed-9246-2b5ebef623ad,2023-04-15 21:42:00,Delo.si,2,Člani komisije se niso opredeljevali do znakov...
3,01e591ff-d2b1-11ed-83eb-0bd28cf4c8f0,2023-04-04 00:00:00,Jana,3,"žJanimn jdrawiljfta Unija f 0 , Težave s ščit..."
4,03e9491b-dd47-11ed-9246-2b5ebef623ad,2023-04-17 19:37:00,Zurnal24.si,4,"""Tudi vsi v zdravstvenem sistemu bodo dobili s..."
...,...,...,...,...,...
95,34ce443c-d2da-11ed-9f09-af460eec62b1,2023-04-04 13:16:00,Sta.si,44,Prodaja prehranskih dopolnil je vse bolj razši...
96,35d75476-e0da-11ed-9f09-af460eec62b1,2023-04-22 08:50:00,Rtvslo.si,96,Raziskujemo spanje. Vsi že poznamo spalne cikl...
97,35e5b4e9-d511-11ed-9f09-af460eec62b1,2023-04-07 08:53:00,Siol.net,97,"Termometer prikaže, kako vroč je članek. Skupn..."
98,3632728f-d881-11ed-bfff-ed625638e8e4,2023-04-11 15:31:00,24ur.com,59,SLOVENIJA\n'Parkinsonova bolezen v našem siste...
